<a href="https://colab.research.google.com/github/doctorpeppercokebear/image_classification/blob/main/%EC%9D%B4%EB%AF%B8%EC%A7%80_%EC%A6%9D%EA%B0%95_%EB%B0%8F_%EB%8D%B0%EC%9D%B4%ED%84%B0_%EC%A0%84%EC%B2%98%EB%A6%AC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### 구글 드라이브 연결

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### 데이터 압축 풀기

In [ ]:
!unzip -q "/content/drive/MyDrive/PetHealthCheck/data/개_안구_squared_all.zip"


### train, val, test 나누기

In [ ]:
!pip install split-folders   # splitfolders install

In [ ]:
import splitfolders

splitfolders.ratio('/content/CVProject/개_안구_squared_all', output="dog_eye_tr_te_va_squared_all_5class_622", ratio=(0.6, 0.2, 0.2))  # train/val/test = 6:2:2

Copying files: 198874 files [01:08, 2909.95 files/s]


### 필요없는 클래스 삭제

In [ ]:
import os
import shutil

# 삭제할 파일이 있는 디렉토리 경로
base_path = "/content/dog_eye_tr_te_va_squared_all_5class_622/train"

# 삭제할 파일들의 이름 리스트
files_to_delete = [
    "결막염",
    "비궤양성각막질환",
    "색소침착성각막염",
    "안검내반증",
    "안검염",
    "유루증"
]

# 파일 삭제
for file_name in files_to_delete:
    file_path = os.path.join(base_path, file_name)
    if os.path.exists(file_path):
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_name} 파일을 삭제했습니다.")
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
            print(f"{file_name} 디렉토리를 삭제했습니다.")
    else:
        print(f"{file_name} 파일이 존재하지 않습니다.")

결막염 디렉토리를 삭제했습니다.
비궤양성각막질환 디렉토리를 삭제했습니다.
색소침착성각막염 디렉토리를 삭제했습니다.
안검내반증 디렉토리를 삭제했습니다.
안검염 디렉토리를 삭제했습니다.
유루증 디렉토리를 삭제했습니다.


In [ ]:
import os
import shutil

# 삭제할 파일이 있는 디렉토리 경로
base_path = "/content/dog_eye_tr_te_va_squared_all_5class_622/val"

# 삭제할 파일들의 이름 리스트
files_to_delete = [
    "결막염",
    "비궤양성각막질환",
    "색소침착성각막염",
    "안검내반증",
    "안검염",
    "유루증"
]

# 파일 삭제
for file_name in files_to_delete:
    file_path = os.path.join(base_path, file_name)
    if os.path.exists(file_path):
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_name} 파일을 삭제했습니다.")
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
            print(f"{file_name} 디렉토리를 삭제했습니다.")
    else:
        print(f"{file_name} 파일이 존재하지 않습니다.")

결막염 디렉토리를 삭제했습니다.
비궤양성각막질환 디렉토리를 삭제했습니다.
색소침착성각막염 디렉토리를 삭제했습니다.
안검내반증 디렉토리를 삭제했습니다.
안검염 디렉토리를 삭제했습니다.
유루증 디렉토리를 삭제했습니다.


In [ ]:
import os
import shutil

# 삭제할 파일이 있는 디렉토리 경로
base_path = "/content/dog_eye_tr_te_va_squared_all_5class_622/test"

# 삭제할 파일들의 이름 리스트
files_to_delete = [
    "결막염",
    "비궤양성각막질환",
    "색소침착성각막염",
    "안검내반증",
    "안검염",
    "유루증"
]

# 파일 삭제
for file_name in files_to_delete:
    file_path = os.path.join(base_path, file_name)
    if os.path.exists(file_path):
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_name} 파일을 삭제했습니다.")
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
            print(f"{file_name} 디렉토리를 삭제했습니다.")
    else:
        print(f"{file_name} 파일이 존재하지 않습니다.")

결막염 디렉토리를 삭제했습니다.
비궤양성각막질환 디렉토리를 삭제했습니다.
색소침착성각막염 디렉토리를 삭제했습니다.
안검내반증 디렉토리를 삭제했습니다.
안검염 디렉토리를 삭제했습니다.
유루증 디렉토리를 삭제했습니다.


### 이미지 증강

In [ ]:
import os
import shutil
from PIL import Image, ImageEnhance
from tqdm import tqdm

# --- 설정 ---
ORI_PATH = '/content/dog_eye_tr_te_va_squared_all_5class_622/train'  # 원본 이미지 폴더 경로
DEST_PATH = '/content/train_augmented_1'  # 증강 이미지 저장 폴더 경로
ALLOWED_EXTENSIONS = ('.png', '.jpg', '.jpeg', '.bmp') # 처리할 이미지 확장자

# --- 메인 로직 ---
# 질병 클래스 목록 가져오기
disease_folders = [d for d in os.listdir(ORI_PATH) if os.path.isdir(os.path.join(ORI_PATH, d))]

# tqdm으로 전체 진행률 표시
for disease in tqdm(disease_folders, desc="Total Progress"):
    if disease == "정상":  # '정상' 클래스는 증강에서 제외
        continue

    source_folder = os.path.join(ORI_PATH, disease)
    dest_folder = os.path.join(DEST_PATH, disease)
    os.makedirs(dest_folder, exist_ok=True) # 대상 폴더 생성 (이미 존재하면 무시)

    # 처리할 파일 목록 미리 생성 및 이미지 개수 출력
    file_list = [f for f in os.listdir(source_folder) if f.lower().endswith(ALLOWED_EXTENSIONS)]
    print(f"\nFound {len(file_list)} images in '{disease}' class.")

    # tqdm으로 클래스별 증강 진행률 표시
    for filename in tqdm(file_list, desc=f"Augmenting {disease}", leave=False):
        try:
            img_path = os.path.join(source_folder, filename)
            img = Image.open(img_path)
            file_name_only, file_ext = os.path.splitext(filename) # 파일 이름과 확장자 분리

            # 1. 원본 이미지 복사
            shutil.copy2(img_path, os.path.join(dest_folder, filename))

            # 2. 다양한 증강 기법 적용 및 저장
            augmentations = {
                'rotated90': img.rotate(90), # 90도 회전
                'rotated180': img.rotate(180), # 180도 회전
                'rotated270': img.rotate(270), # 270도 회전
                'hflip': img.transpose(Image.FLIP_LEFT_RIGHT), # 좌우 반전
                'vflip': img.transpose(Image.FLIP_TOP_BOTTOM), # 상하 반전
                'bright': ImageEnhance.Brightness(img).enhance(1.2), # 밝기 증가
                'contrast': ImageEnhance.Contrast(img).enhance(1.2) # 대비 증가
            }

            for aug_name, aug_img in augmentations.items():
                new_filename = f'{aug_name}_{file_name_only}{file_ext}' # 새 파일 이름 생성 (증강 타입_원본파일이름.확장자)
                aug_img.save(os.path.join(dest_folder, new_filename)) # 증강된 이미지 저장

        except Exception as e:
            print(f"Error processing file {filename}: {e}") # 파일 처리 중 오류 발생 시 출력
            continue # 다음 파일로 건너뛰기

print("\nImage augmentation completed.") # 전체 증강 완료 메시지 출력

Total Progress:   0%|          | 0/5 [00:00<?, ?it/s]


Found 9277 images in '궤양성각막질환' class.



Total Progress:  40%|████      | 2/5 [00:59<01:29, 29.81s/it]


Found 6478 images in '핵경화' class.



Total Progress:  60%|██████    | 3/5 [01:47<01:14, 37.49s/it]


Found 13931 images in '백내장' class.



Total Progress:  80%|████████  | 4/5 [03:31<01:01, 62.00s/it]


Found 3230 images in '안검종양' class.



Total Progress: 100%|██████████| 5/5 [03:54<00:00, 46.88s/it]


Image augmentation completed.


### 데이터 압축해서 구글 드라이브에 저장

In [18]:
# prompt: /content/개_train_test_val_undersample 를  /content/drive/MyDrive/CV_project/data/안구질환  에 zip 파일로 저장하는 코드 알려줘

!zip -r /content/drive/MyDrive/PetHealthCheck/dog_eye_tr_te_va_squared_all_5class_622.zip /content/dog_eye_tr_te_va_squared_all_5class_622

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
  adding: content/dog_eye_tr_te_va_squared_all_5class_622/test/백내장/crop_D11_1ce5eb23-60a5-11ec-8402-0a7404972c70.jpg (deflated 2%)
  adding: content/dog_eye_tr_te_va_squared_all_5class_622/test/백내장/crop_D20_0ea279fe-60a5-11ec-8402-0a7404972c70.jpg (deflated 3%)
  adding: content/dog_eye_tr_te_va_squared_all_5class_622/test/백내장/crop_D58_0dba905c-60a5-11ec-8402-0a7404972c70.jpg (deflated 2%)
  adding: content/dog_eye_tr_te_va_squared_all_5class_622/test/백내장/crop_D13_2f2dc6df-60a5-11ec-8402-0a7404972c70.jpg (deflated 2%)
  adding: content/dog_eye_tr_te_va_squared_all_5class_622/test/백내장/crop_D48_030b922e-60a5-11ec-8402-0a7404972c70.jpg (deflated 2%)
  adding: content/dog_eye_tr_te_va_squared_all_5class_622/test/백내장/crop_D0_0d4dc616-60a5-11ec-8402-0a7404972c70.jpg (deflated 1%)
  adding: content/dog_eye_tr_te_va_squared_all_5class_622/test/백내장/crop_D21_2f946153-60a5-11ec-8402-0a7404972c70.jpg (deflated 2%)
  adding: content/dog_eye_tr_te_va_squared_all_5

In [15]:
import os
import shutil

# 삭제할 파일이 있는 디렉토리 경로
base_path = "/content/dog_eye_tr_te_va_squared_all_5class_622/train"

# 삭제할 파일들의 이름 리스트
files_to_delete = [
    "궤양성각막질환",
    "백내장",
    "안검종양",
    "안검종양",
    "핵경화"

]

# 파일 삭제
for file_name in files_to_delete:
    file_path = os.path.join(base_path, file_name)
    if os.path.exists(file_path):
        if os.path.isfile(file_path):
            os.remove(file_path)
            print(f"{file_name} 파일을 삭제했습니다.")
        elif os.path.isdir(file_path):
            shutil.rmtree(file_path)
            print(f"{file_name} 디렉토리를 삭제했습니다.")
    else:
        print(f"{file_name} 파일이 존재하지 않습니다.")

궤양성각막질환 디렉토리를 삭제했습니다.
백내장 디렉토리를 삭제했습니다.
안검종양 디렉토리를 삭제했습니다.
안검종양 파일이 존재하지 않습니다.
핵경화 디렉토리를 삭제했습니다.




---



---



In [16]:
import os
import glob

base_dir = '/content/dog_eye_tr_te_va_squared_all_5class_622'  # 이미지 폴더 경로

for folder_name in os.listdir(base_dir):
    folder_path = os.path.join(base_dir, folder_name)
    if os.path.isdir(folder_path):
        image_files = glob.glob(os.path.join(folder_path, '*/*.jpg'))  # jpg 이미지만 확인
        num_images = len(image_files)
        print(f"{folder_name}: {num_images} images")


val: 27997 images
train: 314379 images
test: 28005 images


In [17]:
import os

base_dir = '/content/dog_eye_tr_te_va_squared_all_5class_622/train'

for root, dirs, files in os.walk(base_dir):
    if root != base_dir:  # base_dir 자체는 제외
        num_images = sum(1 for file in files if file.endswith('.jpg') or file.endswith('.png'))  # jpg, png 이미지 확인
        print(f"{os.path.basename(root)}: {num_images} images")


정상: 51083 images
궤양성각막질환: 74216 images
핵경화: 51824 images
백내장: 111416 images
안검종양: 25840 images


In [ ]:
# # rotate 붙은 이미지만 다시 vflip 해준다
# path = "/content/train_1/안검종양"
# files = os.listdir(path)

In [ ]:
import os
import shutil
from PIL import Image

ori = '/content/train_1/안검종양'  # 원본 이미지 폴더 경로
dest = '/content/augmented_안검종양'  # 증강 이미지 저장 폴더 경로

# 이미지 파일만 필터링
image_files = [f for f in os.listdir(ori) if not os.path.isdir(os.path.join(ori, f))]
print(f"Found {len(image_files)} images.")

cnt = 0
os.makedirs(dest, exist_ok=True)  # 증강된 이미지 저장 경로 생성

for file in image_files:  # 이미지 파일만 반복
    img_path = os.path.join(ori, file)

    # 파일이 "rotated"로 시작하는 경우에만 처리
    if file.startswith("rotated"):
        try:
            img = Image.open(img_path)
        except (IOError, ValueError):
            print(f"Cannot open image file: {img_path}")
            continue

        file_name, file_ext = os.path.splitext(file)

        # 이미지 증강 및 저장 (좌우, 상하 반전만 수행)
        try:
            img.transpose(Image.FLIP_LEFT_RIGHT).save(os.path.join(dest, f'ro_hflip_{file_name}{file_ext}'))
            img.transpose(Image.FLIP_TOP_BOTTOM).save(os.path.join(dest, f'ro_vflip_{file_name}{file_ext}'))
        except Exception as e:
            print(f"Error augmenting image file: {img_path}, Error: {e}")
            continue

        cnt += 1
        if cnt % 500 == 0:
            print(f"{cnt} files augmented.")

print("Done.")


Found 400 images.
Done.


In [ ]:
# # prompt: /content/train_1/안검종양/안검내반증 이 폴더 삭제하는 코드 알려줘

# shutil.rmtree("/content/train_1/안검종양/안검내반증")


In [ ]:
import os

base_dir = '/content/augmented_안검종양'

for root, dirs, files in os.walk(base_dir):
    if root != base_dir:  # base_dir 자체는 제외
        num_images = sum(1 for file in files if file.endswith('.jpg'))  # jpg, png 이미지 확인
        print(f"{os.path.basename(root)}: {num_images} images")


In [ ]:
# prompt: /content/augmented_안검종양 이 경로의 이미지 개수를 알려줘

base_dir = '/content/augmented_안검종양'

total_images = 0

for root, dirs, files in os.walk(base_dir):
    for file in files:
        if file.endswith('.jpg'):
            total_images += 1

print(f"Total images in '/content/augmented_안검종양': {total_images}")


Total images in '/content/augmented_안검종양': 480


In [ ]:
import os
import shutil

source_dir = "/content/augmented_안검종양"
target_dir = "/content/train_1/안검종양"

# Copy the source directory to the target directory, allowing existing directories to be overwritten
shutil.copytree(source_dir, target_dir, dirs_exist_ok=True)

print("이미지를 성공적으로 병합했습니다.")

이미지를 성공적으로 병합했습니다.


In [ ]:
import os
import shutil

source_dir = "/content/train_1"
target_dir = "/content/개_train_test_val_squared/train"

try:
    # 기존 디렉토리가 존재해도 덮어쓰도록 설정하여 소스 디렉토리를 타겟 디렉토리로 복사
    shutil.copytree(source_dir, target_dir, dirs_exist_ok=True)
    print("이미지를 성공적으로 병합했습니다.")
except Exception as e:
    print(f"이미지 병합 중 오류가 발생했습니다: {e}")


이미지를 성공적으로 병합했습니다.


In [ ]:
import os

base_dir = '/content/개_train_test_val_squared'

for root, dirs, files in os.walk(base_dir):
    if root != base_dir:  # base_dir 자체는 제외
        num_images = sum(1 for file in files if file.endswith('.jpg') or file.endswith('.png'))  # jpg, png 이미지 확인
        print(f"{os.path.basename(root)}: {num_images} images")


test: 0 images
안검내반증: 674 images
비궤양성각막질환: 77 images
궤양성각막질환: 415 images
안검종양: 11 images
백내장: 107 images
색소침착성각막염: 45 images
정상: 8515 images
결막염: 111 images
유루증: 841 images
안검염: 129 images
핵경화: 72 images
train: 0 images
안검내반증: 32337 images
비궤양성각막질환: 3606 images
궤양성각막질환: 19876 images
안검종양: 960 images
백내장: 5100 images
색소침착성각막염: 2122 images
정상: 68111 images
결막염: 5298 images
유루증: 40326 images
안검염: 6126 images
핵경화: 3414 images
val: 0 images
안검내반증: 672 images
비궤양성각막질환: 75 images
궤양성각막질환: 414 images
안검종양: 10 images
백내장: 106 images
색소침착성각막염: 44 images
정상: 8513 images
결막염: 109 images
유루증: 840 images
안검염: 127 images
핵경화: 71 images


In [ ]:
import os
import shutil

# 삭제할 디렉토리 경로
dir_to_delete = '/content/개_train_test_val_squared/val/안검내반증/.ipynb_checkpoints'

# 디렉토리가 존재하는지 확인하고 삭제
if os.path.exists(dir_to_delete) and os.path.isdir(dir_to_delete):
    shutil.rmtree(dir_to_delete)
    print(f"'{dir_to_delete}' 디렉토리가 성공적으로 삭제되었습니다.")
else:
    print(f"'{dir_to_delete}' 디렉토리가 존재하지 않습니다.")


'/content/개_train_test_val_squared/val/안검내반증/.ipynb_checkpoints' 디렉토리가 성공적으로 삭제되었습니다.


In [ ]:
# prompt: /content/개_train_test_val_squared 이 경로의 파일을 /content/drive/MyDrive/CV_project/data/안구질환 이 경로로 zip 파일로 저장하는 코드 알려줘

!zip -r /content/drive/MyDrive/CV_project/data/안구질환/개_train_test_val_augmentation.zip /content/개_train_test_val_squared


In [ ]:
import glob
import random
import os
import shutil

# 샘플링할 클래스 이름들
target_classes = [
    '결막염',
    '궤양성각막질환',
    '백내장',
    '비궤양성각막질환',
    '색소침착성각막염',
    '안검내반증',
    '안검염',
    '유루증',
    '핵경화',
    '정상'
]

# 샘플링할 이미지 개수
target_num_images = 960

# 기본 경로
base_dir = "/content/개_train_test_val_squared/train"

for target_class in target_classes:
    # 대상 디렉토리
    target_dir = os.path.join(base_dir, target_class)

    # 해당 클래스의 모든 이미지 경로를 가져옴
    image_paths = glob.glob(os.path.join(target_dir, "*.jpg")) + glob.glob(os.path.join(target_dir, "*.png"))

    # 이미지 개수가 이미 적은 경우
    if len(image_paths) <= target_num_images:
        print(f"{target_class} 클래스의 이미지가 이미 {target_num_images}개 이하입니다.")
    else:
        # 랜덤하게 이미지를 선택
        selected_images = random.sample(image_paths, target_num_images)

        # 선택된 이미지를 제외한 나머지 이미지를 삭제
        for image_path in image_paths:
            if image_path not in selected_images:
                os.remove(image_path)

        print(f"{target_class} 클래스의 이미지를 {target_num_images}개로 줄였습니다.")


결막염 클래스의 이미지가 이미 960개 이하입니다.
궤양성각막질환 클래스의 이미지가 이미 960개 이하입니다.
백내장 클래스의 이미지가 이미 960개 이하입니다.
비궤양성각막질환 클래스의 이미지가 이미 960개 이하입니다.
색소침착성각막염 클래스의 이미지가 이미 960개 이하입니다.
안검내반증 클래스의 이미지가 이미 960개 이하입니다.
안검염 클래스의 이미지가 이미 960개 이하입니다.
유루증 클래스의 이미지가 이미 960개 이하입니다.
핵경화 클래스의 이미지가 이미 960개 이하입니다.
정상 클래스의 이미지를 960개로 줄였습니다.


In [ ]:
import os

base_dir = '/content/개_train_test_val_squared'

for root, dirs, files in os.walk(base_dir):
    if root != base_dir:  # base_dir 자체는 제외
        num_images = sum(1 for file in files if file.endswith('.jpg') or file.endswith('.png'))  # jpg, png 이미지 확인
        print(f"{os.path.basename(root)}: {num_images} images")


test: 0 images
안검내반증: 674 images
비궤양성각막질환: 77 images
궤양성각막질환: 415 images
안검종양: 11 images
백내장: 107 images
색소침착성각막염: 45 images
정상: 8515 images
결막염: 111 images
유루증: 841 images
안검염: 129 images
핵경화: 72 images
train: 0 images
안검내반증: 960 images
비궤양성각막질환: 960 images
궤양성각막질환: 960 images
안검종양: 960 images
백내장: 960 images
색소침착성각막염: 960 images
정상: 960 images
결막염: 960 images
유루증: 960 images
안검염: 960 images
핵경화: 960 images
val: 0 images
안검내반증: 672 images
비궤양성각막질환: 75 images
궤양성각막질환: 414 images
안검종양: 10 images
백내장: 106 images
색소침착성각막염: 44 images
정상: 8513 images
결막염: 109 images
유루증: 840 images
안검염: 127 images
핵경화: 71 images


In [ ]:
%ls /content/